In [1]:
# import run_search
import run_all
import json
import pickle
import importlib
import helper
import NCBIFetch
import datetime
import NCBISearch
# import run_fetch
# import clean_folder
import PMCParse
import PUBMEDParse
# importlib.reload(PUBMEDParse)
# importlib.reload(PMCParse)
# importlib.reload(NCBIFetch)
importlib.reload(helper)
# # importlib.reload(run_search)
# importlib.reload(run_all)
import newPush
importlib.reload(newPush)

<module 'newPush' from '/Users/sophierand/Desktop/Github/lit_rev_tool/newPush.py'>

In [2]:
con,cur  = helper.connect_to_db()

Connecting to database
host='localhost' dbname= 'lit_rev' user='postgres' password='gres'


In [3]:
input_term = 'syndromic surveillance'
input_db = 'pubmed'
email_id = 'srand1@health.nyc.gov'

In [4]:
run_all.main(input_term,input_db,email_id)

In [5]:
parse_df = helper.id_run('parse')
# parse_df.head()
# parse_df = parse_df_full.head()

In [6]:
push_inst = newPush.push_data(cur,con,parse_df,input_term)

In [7]:
current_id, current_id_tup = push_inst.push_id()

In [8]:
new_df = push_inst.push_detail(current_id,current_id_tup)

In [11]:
title_ins = """INSERT INTO public.datapull_title(
associatedid, pullsource, title, journalname, journaliso, pubtype, publicationdate, pubday, pubmonth, pubyear)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
select currval('datapull_title_unique_id_seq');"""

In [9]:
values_list = []
for index, row in new_df.iterrows():
    associated_id = row['associatedId']
#     associated_id_int = int(associated_id)
    pull_source = row['pullsource']
    title = row['title']
    journal_name = row['journalName']
    journal_iso = row['journalISO']
    pub_type = row['pubtype']
    publication_date = row['publishdatefull']
    try:
        pub_day = row['publishdate']['day']
    except:
        pub_day = None
    try:
        pub_month = row['publishdate']['month']
    except:
        pub_month = None
    try:
        pub_year = row['publishdate']['year']
    except:
        pub_year = None

    values_list.append((associated_id,pull_source,title,journal_name,journal_iso,pub_type,publication_date,pub_day,pub_month,pub_year))
    
# values_list

In [10]:
values_list
args = values_list[1]
args

('26957967',
 'pubmed',
 'Evaluating the use of cell phone messaging for community Ebola syndromic surveillance in high risked settings in Southern Sierra Leone.',
 'African health sciences',
 'Afr Health Sci',
 ['Evaluation Studies', 'Journal Article'],
 '3/10/2016',
 '10',
 '3',
 '2016')

In [12]:
cur.execute(title_ins, args)
pk_id_tup = cur.fetchall()
con.commit()

In [14]:
# pk_id_tup
pk_id = pk_id_tup[0][0]

In [ ]:
parse_df.pullsource.unique().item()

In [ ]:
import PUSH
importlib.reload(PUSH)

In [ ]:
cur_id, cur_id_tup = PUSH.pull_id(cur,con,input_term,input_db)

In [ ]:
cur_id
# cur_id_tup

In [ ]:
# PUSH.pull_detail(cur,con,cur_id_tup,cur_id)

In [ ]:
parse_list_fetch.iloc[0]

In [ ]:
associated_id = parse_list_fetch.iloc[0]['associatedId']
# pub_type = parse_list_fetch.iloc[0]['pubtype']
pull_source = input_db
value_store = 'store'
optional_id_1 = parse_list_fetch.iloc[0]['optionalId01']
optional_id_2 = None
note = None
detail_args = (cur_id, pull_source,associated_id,value_store,note,optional_id_1,optional_id_2)
detail_args

In [ ]:
detail_ins = """INSERT INTO public.datapull_detail(
pullid, pullsource, associatedid, valuestore, note, optionalid01, optionalid02)
VALUES (%s, %s, %s, %s, %s, %s, %s);"""


# cur.execute(detail_ins,detail_args)
# con.commit()


In [ ]:
cur.execute("""SELECT pullsource from public.datapull_id  as a where a.pullid = %s""",cur_id_tup)
cur.fetchall()[0][0]

In [ ]:
unique_id_list

In [ ]:
cur_id, cur_id_tup = PUSH.pull_id(cur,con,input_term,input_db)

In [ ]:
cur.execute("SELECT pullsource, associatedid from public.datapull_detail")
unique_id_list = cur.fetchall()
note = None
values_list = []
for index, row in parse_df.iterrows():
    pull_id = cur_id
    associated_id = row['associatedId']
    associated_id_int = int(associated_id)
    optional_id_1 = row['optionalId01']
    optional_id_2 = row['optionalId02']
#     cur.execute("""SELECT pullsource from public.datapull_id  as a where a.pullid = %s""",cur_id_tup)
#     pull_source = cur.fetchall()[0][0]
    if (pull_source,associated_id) not in unique_id_list:
        value_store = 'store'
    elif (pull_source,associated_id) in unique_id_list:
        value_store = 'duplicate'
        parse_df.drop(index, inplace = TRUE)
    values_list.append((pull_id,pull_source,associated_id,value_store,note,optional_id_1,optional_id_2)) 

In [ ]:
detail_ins = """INSERT INTO public.datapull_detail(
pullid, pullsource, associatedid, valuestore, note, optionalid01, optionalid02)
VALUES (%s, %s, %s, %s, %s, %s, %s);"""

In [ ]:
values_list

In [ ]:
cur.executemany(detail_ins,values_list)


In [ ]:
cur.execute("SELECT max(pullid) as NewID from public.datapull_id;") 

In [ ]:
cur.execute("SELECT pullsource, associatedid from public.datapull_detail")
cur.fetchall()

In [ ]:
cur.execute("SELECT * from public.datapull_detail")
cur.fetchall()

In [ ]:
import PUSH
importlib.reload(PUSH)

In [ ]:
PUSH.pull_detail(cur = cur,con = con,current_id_tup=cur_id_tup,current_id=cur_id,parse_df = parse_df)

In [ ]:
current_id_tup = cur_id_tup

In [ ]:
cur.execute("""SELECT pullsource from public.datapull_id  as a where a.pullid = %s""",current_id_tup)
pull_source = cur.fetchall()[0][0]

In [ ]:
pull_source

In [ ]:
cur_id

In [ ]:
parse_df

In [ ]:
cur.execute("SELECT max(pullid) as NewID from public.datapull_id;") 
cur_id_tup = cur.fetchall()
cur_id = cur_id_tup[0][0]